In [2]:
import pandas as pd

In [1]:
import torch

In [3]:
comments = pd.read_excel('Sub-I equity eval-comments-20230907.xlsx')

In [4]:
subICommets = pd.read_excel('Sub-I equity eval-comments-20230907.xlsx', 'Sub-I comments')

In [5]:
studentInfo = pd.read_excel('Sub-I equity eval-20230526.xlsx')
studentInfo = studentInfo.set_index('DEid')

In [6]:
StudentIDs = comments['DeID'].unique()

In [7]:
print(len(StudentIDs))

1473


In [8]:
print(studentInfo.index.values)

['SOG' 'SOJ' 'SOK' ... 'BAKS' 'BAJZ' 'BAJL']


In [10]:
from transformers import AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

/Users/keshen/opt/anaconda3/envs/UCSF/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")
print(model.config)
print(model.classifier)  # Access the existing classifier layers

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Linear(in_features=768, out_features=2, bias=True)


In [11]:
dataset = load_dataset("glue", "sst2")
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", device='cpu')

In [12]:
max_sequence_length = 128

In [13]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=max_sequence_length)

In [14]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map: 100%|███████████████████████████| 872/872 [00:00<00:00, 5591.33 examples/s]


In [15]:
class CustomDataCollator:
    def __call__(self, batch):
        input_ids = torch.stack([example["input_ids"] for example in batch])
        attention_mask = torch.stack([example["attention_mask"] for example in batch])
        labels = torch.tensor([example["label"] for example in batch])
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
data_collator = CustomDataCollator()

In [16]:
training_args = TrainingArguments(
    output_dir="./sst2_sentiment_finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
    save_steps=100,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    remove_unused_columns=False,
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  # Use the custom data collator
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [18]:
!export PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.504000,0.553218
200,0.595800,0.503988
300,0.443800,1.091465
400,0.620400,0.907501
500,0.533700,0.554187
600,0.500900,0.803412
700,0.578600,0.816200
800,1.048800,0.797697
900,0.611400,0.519864
1000,0.695400,0.654756


TrainOutput(global_step=50514, training_loss=0.6831287103371954, metrics={'train_runtime': 59164.1087, 'train_samples_per_second': 3.415, 'train_steps_per_second': 0.854, 'total_flos': 1.329019985058048e+16, 'train_loss': 0.6831287103371954, 'epoch': 3.0})

In [20]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 0.6976110935211182, 'eval_runtime': 20.551, 'eval_samples_per_second': 42.431, 'eval_steps_per_second': 10.608, 'epoch': 3.0}


In [21]:
trainer.save_model("./SST-2-pubmedBert.model")

In [ ]:

# from datasets import load_dataset
# from torch.utils.data import DataLoader
# import transformers
# from transformers import pipeline
# # model_name = "./SST-2-pubmedBert.model"

# model = AutoModelForSequenceClassification.from_pretrained(model_name)

# tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", device='cpu')

# Valdataset = load_dataset("glue", "sst2", split="validation")

# # Define a function for tokenizing and padding the dataset
# def tokenize_and_pad(examples, padding="max_length", max_length=64):
#     inputs = tokenizer(
#         examples["sentence"],
#         padding=padding,
#         truncation=True,
#         return_tensors="pt",
#         max_length=max_length,
#     )
#     return inputs

# tokenized_dataset = Valdataset.map(tokenize_and_pad, batched=True)
# tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# data_loader = DataLoader(tokenized_dataset, batch_size=8, num_workers=4)

# results = []
# for batch in data_loader:
#     with torch.no_grad():
#         inputs = {
#             "input_ids": batch["input_ids"],
#             "attention_mask": batch["attention_mask"],
#             "labels": batch["label"]
#         }
#         outputs = model(**inputs)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=1)
#         results.extend(predictions.cpu().numpy())

# print("Predictions:", results)




# input_text = "Continue to work on physical exam skills including cardiac auscultation."
# input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# with torch.no_grad():
#     outputs = model(**input_ids)

# logits = outputs.logits
# probabilities = logits.softmax(dim=1)  
# print(probabilities)
# predicted_class_index = logits.argmax(1)

# class_labels = ["Negative", "Positive"] 
# predicted_class_label = class_labels[predicted_class_index.item()]

# print("Predicted Class Label:", predicted_class_label)

In [9]:
import transformers
from transformers import pipeline
model = transformers.pipeline('sentiment-analysis', model='finiteautomata/bertweet-base-sentiment-analysis', top_k=None)
model("I got the sense that XX was really nervous about being an M3 and interacting with patients and the rest of the team. This is not unreasonable given that this was his very first rotation, but it may pose some difficulty with his interactions with patients and the medical team. It may be helpful for him to work on some strategies to minimize nerves and to remember that in a really closely supervised learning environment he's not going to hurt anyone.")[0]
# https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

/Users/keshen/opt/anaconda3/envs/UCSF/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'label': 'NEU', 'score': 0.8738716840744019},
 {'label': 'NEG', 'score': 0.10971225798130035},
 {'label': 'POS', 'score': 0.01641611009836197}]

In [10]:
print(studentInfo.index.values)

['SOG' 'SOJ' 'SOK' ... 'BAKS' 'BAJZ' 'BAJL']


In [11]:
print(comments.head())

   DeID core_year  bridges_ind    Course           Comment type  \
0  BAAA      2019            1  ANES 110  Constructive Comments   
1  BAAA      2019            1  ANES 110  Constructive Comments   
2  BAAA      2019            1  ANES 110  Constructive Comments   
3  BAAA      2019            1  ANES 110  Constructive Comments   
4  BAAA      2019            1  ANES 110  Grade Explanation       

                                             Comment  
0  Continue to keep reading and learning you are ...  
1                 Great job! Keep up the good work!   
2                                                 Na  
3  Awesome job.  Keep up the good work.  And thin...  
4  This grade is based upon attendance at all req...  


### Constructive Comments (clerkship)

In [12]:
InfoDict = {'ANES 110': 'anes_grade', 'FCM 110': 'fcm_grade', 'MED 110': 'med_grade', 'NEURO 110': 'neuro_grade', 'OBGYN 110': 'obgyn_grade',
           'PEDS 110': 'peds_grade', 'PSYCH 110': 'psych_grade', 'SURG 110': 'surg_grade'}
GradeDict = {'F': 0, 'I': 0, 'P': 1, 'H': 2}

In [13]:
def calculateTopK(scores, perc):
    sorted_scores = sorted(scores, reverse=True)
    top_percent = perc
    index_position = int(len(sorted_scores) * top_percent)
    top_scores = sorted_scores[:index_position]
    return top_scores

In [27]:
import datetime
comments['Comment'].fillna('', inplace=True)
LetterGrade = {}
SentiGrade = {}
for deid in studentInfo.index.values:
    filteredComment = comments[(comments['DeID']==deid) & (comments['Comment type']=='Constructive Comments')]
    for index in range(len(filteredComment)):
        row = filteredComment.iloc[index]
        LetterGrade[f"{deid}_{row['Course']}"] = studentInfo.loc[deid][InfoDict[row['Course']]]
        chunk_size = 128
        comment = row['Comment']
        scores = []
        if isinstance(comment, str)  and len(comment.split(' ')) >= 4: 
            chunks = [comment[i:i+chunk_size] for i in range(0, len(comment), chunk_size)]
            chunk_scores = [model([chunk]) for chunk in chunks]
            sentiScore = [s_['score'] for S in chunk_scores for s in S for s_ in s if s_['label'] == 'POS']
            scores = calculateTopK(sentiScore, 0.85)
        if scores:
            SentiGrade[f"{deid}_{row['Course']}"] = sum(scores) / len(scores)
print(SentiGrade)
print(LetterGrade)

{'SOG_ANES 110': 0.9516103267669678, 'SOG_FCM 110': 0.7910568356513977, 'SOG_MED 110': 0.22423385083675385, 'SOG_NEURO 110': 0.6352628663182258, 'SOG_OBGYN 110': 0.9828891754150391, 'SOG_PSYCH 110': 0.991263210773468, 'SOJ_FCM 110': 0.812597143650055, 'SOJ_NEURO 110': 0.38909053057432175, 'SOJ_OBGYN 110': 0.9835836291313171, 'SOJ_PSYCH 110': 0.9871303041776022, 'SOJ_SURG 110': 0.9920054078102112, 'SOK_FCM 110': 0.9136601537466049, 'SOK_MED 110': 0.1413814276456833, 'SOK_NEURO 110': 0.5288213528692722, 'SOK_OBGYN 110': 0.7491739392280579, 'SOK_PSYCH 110': 0.9631905953089396, 'SOK_SURG 110': 0.9896911382675171, 'SOZ_FCM 110': 0.9594930648803711, 'SOZ_MED 110': 0.451509028673172, 'SOZ_NEURO 110': 0.3883574952681859, 'SOZ_OBGYN 110': 0.9455534815788269, 'SOZ_PEDS 110': 0.7607098420461019, 'SOZ_PSYCH 110': 0.9909298419952393, 'SOZ_SURG 110': 0.5721107125282288, 'SPB_FCM 110': 0.9885919988155365, 'SPB_MED 110': 0.24314879377683005, 'SPB_NEURO 110': 0.3568107457831502, 'SPB_OBGYN 110': 0.0953

In [14]:
def getLetterNSentiGrade(commentType, threshold=0.85):
    """
        commentType: 'Constructive Comments' or 'Summary Comments'
    """
    LetterGrade = {}
    SentiGrade = {}
    for deid in studentInfo.index.values:
        filteredComment = comments[(comments['DeID']==deid) & (comments['Comment type']==commentType)]
        for index in range(len(filteredComment)):
            row = filteredComment.iloc[index]
            LetterGrade[f"{deid}_{row['Course']}"] = studentInfo.loc[deid][InfoDict[row['Course']]]
            chunk_size = 128
            comment = row['Comment']
            scores = []
            if isinstance(comment, str)  and len(comment.split(' ')) >= 4: 
                chunks = [comment[i:i+chunk_size] for i in range(0, len(comment), chunk_size)]
                chunk_scores = [model([chunk]) for chunk in chunks]
                sentiScore = [s_['score'] for S in chunk_scores for s in S for s_ in s if s_['label'] == 'POS']
                scores = calculateTopK(sentiScore, threshold)
            if scores:
                SentiGrade[f"{deid}_{row['Course']}"] = sum(scores) / len(scores)
    return LetterGrade, SentiGrade
        

In [15]:
LetterGrade, SentiGrade = getLetterNSentiGrade('Constructive Comments')

In [27]:
for c in InfoDict.keys():
    class_LetterGrade = {key: value for key, value in LetterGrade.items() if c in key}
    class_SentiGrade = {key: value for key, value in SentiGrade.items() if c in key}
    print(c, len(class_LetterGrade))
    print(generalCorrCal(class_LetterGrade, class_SentiGrade, GradeDict, InfoDict))
    print('---------------')


ANES 110 1281
(0.10923409764236987, 0.000579157394475147)
---------------
FCM 110 1464
(0.10584299762100204, 4.955098579051369e-05)
---------------
MED 110 1433
(-0.03235539791508573, 0.2518628104167212)
---------------
NEURO 110 1466
(0.10527127282400185, 5.4762374189954225e-05)
---------------
OBGYN 110 1465
(0.2328584676295938, 2.898338060133732e-18)
---------------
PEDS 110 1387
(0.12923603699575253, 2.071533576649581e-06)
---------------
PSYCH 110 1468
(-0.12540961652452393, 1.4597934044320338e-06)
---------------
SURG 110 1371
(0.10818541731186389, 0.00025751368605776275)
---------------


In [24]:
from scipy import stats

In [25]:
def generalCorrCal(LetterGrade, SentiGrade, GradeDict, InfoDict):
    LG = []
    SG = []
    for deid in studentInfo.index.values:
        for c in InfoDict.keys():
            if f"{deid}_{c}" in LetterGrade and f"{deid}_{c}" in SentiGrade and LetterGrade[f"{deid}_{c}"] in GradeDict:
                LG.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                SG.append(SentiGrade[f"{deid}_{c}"])
    res = stats.spearmanr(SG, LG)
    return res.statistic, res.pvalue

In [31]:
print(generalCorrCal(LetterGrade, SentiGrade, GradeDict, InfoDict))

(0.08272068549350828, 2.2173637210916353e-17)


In [29]:
def partialCorrCal(LetterGrade, SentiGrade, GradeDict, InfoDict):
    LG_UIM = []
    SG_UIM = []
    LG_NONUIM = []
    SG_NONUIM = []
    for deid in studentInfo.index.values:
        for c in InfoDict.keys():
            if f"{deid}_{c}" in LetterGrade and f"{deid}_{c}" in SentiGrade and LetterGrade[f"{deid}_{c}"] in GradeDict:
                if studentInfo.loc[deid]['isURM'] == 1:
                    LG_UIM.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                    SG_UIM.append(SentiGrade[f"{deid}_{c}"])
                elif studentInfo.loc[deid]['isURM'] == 0:
                    LG_NONUIM.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                    SG_NONUIM.append(SentiGrade[f"{deid}_{c}"])
    
    res_UIM = stats.spearmanr(SG_UIM, LG_UIM)
    res_NONUIM = stats.spearmanr(SG_NONUIM, LG_NONUIM)
    print(f"UIM group: {res_UIM.statistic}, {res_UIM.pvalue}; NonUIM group: {res_NONUIM.statistic}, {res_NONUIM.pvalue}.")
    return SG_UIM, SG_NONUIM

In [31]:
for c in InfoDict.keys():
    class_LetterGrade = {key: value for key, value in LetterGrade.items() if c in key}
    class_SentiGrade = {key: value for key, value in SentiGrade.items() if c in key}
    print(c, len(class_LetterGrade))
    SG_UIM, SG_NONUIM = partialCorrCal(class_LetterGrade, class_SentiGrade, GradeDict, InfoDict)
    print(sum(SG_UIM) / len(SG_UIM), sum(SG_NONUIM) / len(SG_NONUIM))
    print(stats.ttest_ind(SG_UIM, SG_NONUIM))
    print('---------------')


ANES 110 1281
UIM group: 0.11829237092090064, 0.034982853573794624; NonUIM group: 0.12825296626865187, 0.0011999565346707062.
0.7169203856659359 0.7245886364169064
TtestResult(statistic=-0.3505253803478716, pvalue=0.7260220715830968, df=951.0)
---------------
FCM 110 1464
UIM group: 0.13767387435196876, 0.00393271324149687; NonUIM group: 0.08131198866410722, 0.012638673823659614.
0.733884567772295 0.747249073881331
TtestResult(statistic=-0.8312630374097294, pvalue=0.40596937577596925, df=1375.0)
---------------
MED 110 1433
UIM group: -0.027295212043478005, 0.5953289466804133; NonUIM group: -0.007598446553789603, 0.8305121560516608.
0.4677918885152846 0.4529374747515048
TtestResult(statistic=0.7854374799563006, pvalue=0.43235545750011806, df=1175.0)
---------------
NEURO 110 1466
UIM group: 0.12140562266044112, 0.01108327369737014; NonUIM group: 0.11336924947716322, 0.0005003191877896889.
0.5715500595471434 0.536458414615475
TtestResult(statistic=2.331407757140701, pvalue=0.01987584665

In [36]:
SG_UIM, SG_NONUIM = partialCorrCal(LetterGrade, SentiGrade, GradeDict, InfoDict)

UIM group: 0.10425908609154368, 3.964775131811991e-09; NonUIM group: 0.07821518191762289, 1.4115225952935828e-10.


In [37]:
print(sum(SG_UIM) / len(SG_UIM), sum(SG_NONUIM) / len(SG_NONUIM))
stats.ttest_ind(SG_UIM, SG_NONUIM)

0.6789154141753295 0.678098848889926


TtestResult(statistic=0.12057081459823452, pvalue=0.9040334389531113, df=9878.0)

### Summary Comments (clerkship)

In [16]:
LetterGrade_SC, SentiGrade_SC = getLetterNSentiGrade('Summary Comments', threshold=1)

In [32]:
for c in InfoDict.keys():
    class_LetterGrade = {key: value for key, value in LetterGrade_SC.items() if c in key}
    class_SentiGrade = {key: value for key, value in SentiGrade_SC.items() if c in key}
    print(c, len(class_LetterGrade))
    print(generalCorrCal(class_LetterGrade, class_SentiGrade, GradeDict, InfoDict))
    print('---------------')


ANES 110 1433
(0.015289055950189869, 0.5634740825149775)
---------------
FCM 110 1454
(0.1839122360172434, 1.5814322206977907e-12)
---------------
MED 110 1469
(0.04044240251910786, 0.12256885004017129)
---------------
NEURO 110 1466
(0.14715035470720492, 1.5641325955049443e-08)
---------------
OBGYN 110 1465
(0.11737023021177029, 6.713226284904191e-06)
---------------
PEDS 110 1397
(0.24555508623831213, 1.369421231741539e-20)
---------------
PSYCH 110 1468
(0.26979260647816083, 7.234570853387261e-26)
---------------
SURG 110 1467
(0.17595513904508436, 1.2046804737841133e-11)
---------------


In [33]:
for c in InfoDict.keys():
    class_LetterGrade = {key: value for key, value in LetterGrade_SC.items() if c in key}
    class_SentiGrade = {key: value for key, value in SentiGrade_SC.items() if c in key}
    print(c, len(class_LetterGrade))
    SG_UIM, SG_NONUIM = partialCorrCal(class_LetterGrade, class_SentiGrade, GradeDict, InfoDict)
    print(sum(SG_UIM) / len(SG_UIM), sum(SG_NONUIM) / len(SG_NONUIM))
    print(stats.ttest_ind(SG_UIM, SG_NONUIM))
    print('---------------')

ANES 110 1433
UIM group: 0.0014022483772940036, 0.976708459067744; NonUIM group: 0.01814362355711931, 0.5791074830414917.
0.7272788362772545 0.7485234851398211
TtestResult(statistic=-1.9341364549257882, pvalue=0.05330192774337792, df=1371.0)
---------------
FCM 110 1454
UIM group: 0.24660257534092275, 2.024094347726229e-07; NonUIM group: 0.171892037955508, 1.2528141425430578e-07.
0.722712124041924 0.7159522544721657
TtestResult(statistic=1.0188819134781302, pvalue=0.3084395804126804, df=1365.0)
---------------
MED 110 1469
UIM group: 0.07978938309157349, 0.09728626296185977; NonUIM group: 0.03261003256727388, 0.31792068288299813.
0.6586764203835688 0.6561995803650932
TtestResult(statistic=0.2750851820996517, pvalue=0.7832922447136057, df=1371.0)
---------------
NEURO 110 1466
UIM group: 0.13126688530931416, 0.005993889190235812; NonUIM group: 0.16936606206156365, 1.7820005416109966e-07.
0.7030531866947807 0.7016281935750296
TtestResult(statistic=0.13403829893661184, pvalue=0.8933919192

In [40]:
print(generalCorrCal(LetterGrade_SC, SentiGrade_SC, GradeDict, InfoDict))

(0.13646048970459157, 2.615549778892272e-49)


In [41]:
SGSC_UIM, SGSC_NONUIM = partialCorrCal(LetterGrade_SC, SentiGrade_SC, GradeDict, InfoDict)

UIM group: 0.13069553473659243, 1.1009904771527576e-14; NonUIM group: 0.14078513858703615, 2.4465936994162498e-34.


In [42]:
print(sum(SGSC_UIM) / len(SGSC_UIM), sum(SGSC_NONUIM) / len(SGSC_NONUIM))
stats.ttest_ind(SGSC_UIM, SGSC_NONUIM)

0.6917773560173544 0.7001514347079294


TtestResult(statistic=-2.2276293010169064, pvalue=0.02592552786154597, df=10927.0)

### Constructive Comments (sub-intern)

In [36]:
SubIGradeDict = {'P': 1, 'H': 2}
SubIInfoDict = {'FCM 140.40': 'subinternship', 'MED 140.01': 'subinternship', 'MED 140.01A': 'subinternship', 'MED 140.01B': 'subinternship', 'MED 140.01C': 'subinternship'}

In [18]:
subICommets['Comment'].fillna('', inplace=True)

In [19]:
def getSubILetterNSentiGrade(commentType):
    """
        commentType: 'Constructive Comments' or 'Summary Comments'
    """
    LetterGrade = {}
    SentiGrade = {}
    for deid in studentInfo.index.values:
        gradesubinterncls = studentInfo.loc[deid]['subinternship']

        filteredComment = subICommets[(subICommets['DeID']==deid) & (subICommets['Comment type']==commentType) & (subICommets['Course']==gradesubinterncls)]
        # subinterncls = filteredComment['Course'].unique().tolist()
        LetterGrade[deid] = studentInfo.loc[deid]['subint_grade']
        chunk_size = 128
        for comment in filteredComment['Comment'].values.tolist():
            if isinstance(comment, str)  and len(comment.split(' ')) >= 4:
                chunks = [comment[i:i+chunk_size] for i in range(0, len(comment), chunk_size)]
                chunk_scores = [model([chunk]) for chunk in chunks]
                sentiScore = [s_['score'] for S in chunk_scores for s in S for s_ in s if s_['label'] == 'POS']
                if sentiScore and studentInfo.loc[deid]['subint_grade'] in SubIGradeDict:
                    SentiGrade[deid] = sum(sentiScore) / len(sentiScore)
    return LetterGrade, SentiGrade

In [20]:
subILetterGradeCC, subISentiGradeCC = getSubILetterNSentiGrade('Constructive Comments')

In [51]:
for c in SubIInfoDict.keys():
    class_LetterGrade = {key: value for key, value in subILetterGradeCC.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    class_SentiGrade = {key: value for key, value in subISentiGradeCC.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    print(c, len(class_LetterGrade))

    print(generalCorrCalSubI(class_LetterGrade, class_SentiGrade, SubIGradeDict))
    # print('---------------')

FCM 140.40 133
(0.06000836607992369, 0.5096875949349686)
MED 140.01 774
(0.11642393888597152, 0.0013325758041156598)
MED 140.01A 217
(0.027149555781501508, 0.7115006411557157)
MED 140.01B 148
(-0.02197779364304909, 0.8054957666566086)
MED 140.01C 167
(-0.12113265626973949, 0.36505323055104366)


In [53]:
for c in SubIInfoDict.keys():
    class_LetterGrade = {key: value for key, value in subILetterGradeCC.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    class_SentiGrade = {key: value for key, value in subISentiGradeCC.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    print(c, len(class_LetterGrade))
    SG_UIM, SG_NONUIM = partialCorrCalSubI(class_LetterGrade, class_SentiGrade, GradeDict)
    print(sum(SG_UIM) / len(SG_UIM), sum(SG_NONUIM) / len(SG_NONUIM))
    print(stats.ttest_ind(SG_UIM, SG_NONUIM))
    print('---------------')

FCM 140.40 133
UIM group: -0.0345834250622541, with p-value 0.8236421281345631; NonUIM group: 0.14537782176138217, with p-value 0.20112459062573365.
0.48960587740635925 0.482457745938842
TtestResult(statistic=0.15361074201180763, pvalue=0.8781725606433394, df=121.0)
---------------
MED 140.01 774
UIM group: 0.21967751639039512, with p-value 0.0007340430164618938; NonUIM group: 0.10664652973980664, with p-value 0.019690938042364308.
0.49376488911311645 0.43113606027289425
TtestResult(statistic=2.559633362367031, pvalue=0.010684553327068491, df=709.0)
---------------
MED 140.01A 217
UIM group: 0.03429810303809886, with p-value 0.8230141371326696; NonUIM group: 0.03503539944392089, with p-value 0.6855271160319945.
0.2234810814651212 0.23926630991310613
TtestResult(statistic=-0.49336522938996835, pvalue=0.6223593004815713, df=179.0)
---------------
MED 140.01B 148
UIM group: -0.2704485438316892, with p-value 0.12795100928152026; NonUIM group: 0.08372489242911722, with p-value 0.45455886920

In [48]:
def generalCorrCalSubI(LetterGrade, SentiGrade, GradeDict):
    LG = []
    SG = []
    for deid in studentInfo.index.values:
        if deid in LetterGrade and deid in SentiGrade and LetterGrade[deid] in GradeDict:
            LG.append(GradeDict[LetterGrade[deid]])
            SG.append(SentiGrade[deid])
    res = stats.spearmanr(SG, LG)
    return res.statistic, res.pvalue

In [61]:
generalCorrCalSubI(subILetterGradeCC, subISentiGradeCC, SubIGradeDict)

(0.11695798345175619, 3.370853995652323e-05)

In [52]:
def partialCorrCalSubI(LetterGrade, SentiGrade, GradeDict):
    LG_UIM = []
    SG_UIM = []
    LG_NONUIM = []
    SG_NONUIM = []
    for deid in studentInfo.index.values:
        if deid in LetterGrade and deid in SentiGrade and LetterGrade[deid] in GradeDict:
            if studentInfo.loc[deid]['isURM'] == 1:
                LG_UIM.append(GradeDict[LetterGrade[deid]])
                SG_UIM.append(SentiGrade[deid])
            elif studentInfo.loc[deid]['isURM'] == 0:
                LG_NONUIM.append(GradeDict[LetterGrade[deid]])
                SG_NONUIM.append(SentiGrade[deid])
    
    res_UIM = stats.spearmanr(SG_UIM, LG_UIM)
    res_NONUIM = stats.spearmanr(SG_NONUIM, LG_NONUIM)
    print(f"UIM group: {res_UIM.statistic}, with p-value {res_UIM.pvalue}; NonUIM group: {res_NONUIM.statistic}, with p-value {res_NONUIM.pvalue}.")
    return SG_UIM, SG_NONUIM

In [65]:
SubI_SG_UIM, SubI_SG_NONUIM = partialCorrCalSubI(subILetterGradeCC, subISentiGradeCC, SubIGradeDict)

UIM group: 0.18389047315388116, with p-value 0.000325521373006547; NonUIM group: 0.10986251397165415, with p-value 0.001858580232102163.


In [67]:
print(sum(SubI_SG_UIM) / len(SubI_SG_UIM), sum(SubI_SG_NONUIM) / len(SubI_SG_NONUIM))
stats.ttest_ind(SubI_SG_UIM, SubI_SG_NONUIM)

0.43189156229896425 0.39233648906500257


TtestResult(statistic=2.156326542754707, pvalue=0.03126041917343325, df=1176.0)

### Summary Comments (sub-intern)

In [21]:
subILetterGrade, subISentiGrade = getSubILetterNSentiGrade('Summary Comments')

In [54]:
for c in SubIInfoDict.keys():
    class_LetterGrade = {key: value for key, value in subILetterGrade.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    class_SentiGrade = {key: value for key, value in subISentiGrade.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    print(c, len(class_LetterGrade))

    print(generalCorrCalSubI(class_LetterGrade, class_SentiGrade, SubIGradeDict))
    # print('---------------')

FCM 140.40 133
(-0.1009484904631257, 0.2665759593224055)
MED 140.01 774
(-0.05028483730748996, 0.1669389393548187)
MED 140.01A 217
(-0.18115332246935853, 0.008666153138297387)
MED 140.01B 148
(-0.07727464290886339, 0.36068405656324976)
MED 140.01C 167
(0.034756601374298944, 0.6646183480514711)


In [56]:
for c in SubIInfoDict.keys():
    class_LetterGrade = {key: value for key, value in subILetterGrade.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    class_SentiGrade = {key: value for key, value in subISentiGrade.items() if subICommets[(subICommets['DeID'] == key) & (subICommets['Course'] == c)]['Course'].unique().size > 0}
    print(c, len(class_LetterGrade))
    SG_UIM, SG_NONUIM = partialCorrCalSubI(class_LetterGrade, class_SentiGrade, GradeDict)
    print(sum(SG_UIM) / len(SG_UIM), sum(SG_NONUIM) / len(SG_NONUIM))
    print(stats.ttest_ind(SG_UIM, SG_NONUIM))
    print('---------------')

FCM 140.40 133
UIM group: 0.14986150860310107, with p-value 0.33157371745448794; NonUIM group: -0.2520913292245244, with p-value 0.025009693286965506.
0.6755890133594984 0.6455524000415287
TtestResult(statistic=0.954634519156056, pvalue=0.34166583947932916, df=121.0)
---------------
MED 140.01 774
UIM group: -0.11991071371743377, with p-value 0.06768297498808767; NonUIM group: -0.025417990096014535, with p-value 0.5793369684126461.
0.6310521606788319 0.6428004222748289
TtestResult(statistic=-1.1650629795516432, pvalue=0.2443853304249059, df=709.0)
---------------
MED 140.01A 217
UIM group: -0.1747655184106389, with p-value 0.2247894016579756; NonUIM group: -0.19740166804352466, with p-value 0.015119204722572303.
0.7609102240922467 0.7516913970281554
TtestResult(statistic=0.5836057170007418, pvalue=0.5601464851120244, df=199.0)
---------------
MED 140.01B 148
UIM group: -0.19197665705766642, with p-value 0.2619979864930259; NonUIM group: -0.08221474501428261, with p-value 0.433360083421

In [69]:
generalCorrCalSubI(subILetterGrade, subISentiGrade, SubIGradeDict)

(-0.07934199264594313, 0.0031183871851880826)

In [70]:
SubI_SG_UIM_SC, SubI_SG_NONUIM_SC = partialCorrCalSubI(subILetterGrade, subISentiGrade, SubIGradeDict)

UIM group: -0.11117545747790203, with p-value 0.025626725558383044; NonUIM group: -0.08851200542681512, with p-value 0.007991190390024936.


In [72]:
print(sum(SubI_SG_UIM_SC) / len(SubI_SG_UIM_SC), sum(SubI_SG_NONUIM_SC) / len(SubI_SG_NONUIM_SC))
stats.ttest_ind(SubI_SG_UIM_SC, SubI_SG_NONUIM_SC)

0.6681043420042816 0.6697930205455114


TtestResult(statistic=-0.21408638160911683, pvalue=0.8305133204050672, df=1298.0)

### Pre and Post grading changes

In [75]:
def general_Pre_Post_Corr(LetterGrade, SentiGrade, GradeDict, InfoDict):
    pre_LG = []
    pre_SG = []
    post_LG = []
    post_SG = []
    for deid in studentInfo.index.values:
        for c in InfoDict.keys():
            if f"{deid}_{c}" in LetterGrade and f"{deid}_{c}" in SentiGrade and LetterGrade[f"{deid}_{c}"] in GradeDict:
                if studentInfo.loc[deid]['core_year'] in ['2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018']:
                    pre_LG.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                    pre_SG.append(SentiGrade[f"{deid}_{c}"])
                else:
                    post_LG.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                    post_SG.append(SentiGrade[f"{deid}_{c}"])
    
    resPre = stats.spearmanr(pre_SG, pre_LG)
    resPost= stats.spearmanr(post_SG, post_LG)
    
    print("Corr for pre is ", resPre.statistic, "with p-value: ", resPre.pvalue)
    print("Corr for post is ", resPost.statistic, "with p-value: ", resPost.pvalue)
    
    print("avg. pre SG is: ", sum(pre_SG) / len(pre_SG), "avg. post SG is: ", sum(post_SG) / len(post_SG))
    print(stats.ttest_ind(pre_SG, post_SG))

In [76]:
general_Pre_Post_Corr(LetterGrade, SentiGrade, GradeDict, InfoDict)  # general pre- and post-changing correlation for clerkship constructive comments

Corr for pre is  0.11558948283421384 with p-value:  2.1906831019665204e-16
Corr for post is  0.03789255044060811 with p-value:  0.005085046317226907
avg. pre SG is:  0.6875077858861842 avg. post SG is:  0.6695521760030603
TtestResult(statistic=2.92275346304814, pvalue=0.003476929573790722, df=10478.0)


In [77]:
general_Pre_Post_Corr(LetterGrade_SC, SentiGrade_SC, GradeDict, InfoDict) # general pre- and post-changing correlation for clerkship summary comments

Corr for pre is  -0.05446110655986024 with p-value:  2.8817249957486375e-05
Corr for post is  0.17071503467678814 with p-value:  1.5167887706895337e-38
avg. pre SG is:  0.7616581830839594 avg. post SG is:  0.6331938486107765
TtestResult(statistic=40.37099996388298, pvalue=0.0, df=11592.0)


In [78]:
### Pre v.s. Post Clerk Constructive Comments (UIM v.s. NONUIM)
def partial_Pre_Post_Corr(LetterGrade, SentiGrade, GradeDict, InfoDict):
    preLG_UIM = []
    preSG_UIM = []
    preLG_NONUIM = []
    preSG_NONUIM = []
    postLG_UIM = []
    postSG_UIM = []
    postLG_NONUIM = []
    postSG_NONUIM = []
    for deid in studentInfo.index.values:
        for c in InfoDict.keys():
            if f"{deid}_{c}" in LetterGrade and f"{deid}_{c}" in SentiGrade and LetterGrade[f"{deid}_{c}"] in GradeDict:
                if studentInfo.loc[deid]['core_year'] in ['2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018']:
                    if studentInfo.loc[deid]['isURM'] == 1 :
                        preLG_UIM.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                        preSG_UIM.append(SentiGrade[f"{deid}_{c}"])
                    else:
                        preLG_NONUIM.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                        preSG_NONUIM.append(SentiGrade[f"{deid}_{c}"])
                    
                else:
                    if studentInfo.loc[deid]['isURM'] == 1 :
                        postLG_UIM.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                        postSG_UIM.append(SentiGrade[f"{deid}_{c}"])
                    else:
                        postLG_NONUIM.append(GradeDict[LetterGrade[f"{deid}_{c}"]])
                        postSG_NONUIM.append(SentiGrade[f"{deid}_{c}"])

    resPreUIM = stats.spearmanr(preSG_UIM, preLG_UIM)
    resPreNONUIM = stats.spearmanr(preSG_NONUIM, preLG_NONUIM)
    
    resPostUIM = stats.spearmanr(postSG_UIM, postLG_UIM)
    resPostNONUIM = stats.spearmanr(postSG_NONUIM, postLG_NONUIM)
    
    print("------------Pre------------")
    print("for UIM group: ", resPreUIM.statistic, "with p-value", resPreUIM.pvalue)
    print("for NonUIM group: ", resPreNONUIM.statistic, "with p-value", resPreNONUIM.pvalue)
    print("------------Post------------")
    print("for UIM group: ", resPostUIM.statistic, "with p-value", resPostUIM.pvalue)
    print("for NonUIM group: ", resPostNONUIM.statistic, "with p-value", resPostNONUIM.pvalue)
    
    print("------------UIM v.s. NONUIM------------")
    print("Pre-changing, UIM group sg: ", sum(preSG_UIM) / len(preSG_UIM), "NonUIM group sg: ", sum(preSG_NONUIM) / len(preSG_NONUIM))
    print(stats.ttest_ind(preSG_UIM, preSG_NONUIM))
    
    print("Post-changing, UIM group sg: ", sum(postSG_UIM) / len(postSG_UIM),"NonUIM group sg: ", sum(postSG_NONUIM) / len(postSG_NONUIM))
    print(stats.ttest_ind(postSG_UIM, postSG_NONUIM))

In [79]:
partial_Pre_Post_Corr(LetterGrade, SentiGrade, GradeDict, InfoDict) ### Pre v.s. Post Clerk Constructive Comments (UIM v.s. NONUIM)

------------Pre------------
for UIM group:  0.12989375221875588 with p-value 1.3831917391345238e-06
for NonUIM group:  0.10732689853125245 with p-value 8.330865460445836e-11
------------Post------------
for UIM group:  0.09540586403642991 with p-value 5.0313315499672934e-05
for NonUIM group:  0.014904969050332973 with p-value 0.36701551268421106
------------UIM v.s. NONUIM------------
Pre-changing, UIM group sg:  0.675742682985229 NonUIM group sg:  0.6919386728420215
TtestResult(statistic=-1.645067983221698, pvalue=0.10001848801757361, df=5013.0)
Post-changing, UIM group sg:  0.6813337403935608 NonUIM group sg:  0.6637658687989935
TtestResult(statistic=1.924949511426889, pvalue=0.05428753916803904, df=5463.0)


In [80]:
partial_Pre_Post_Corr(LetterGrade_SC, SentiGrade_SC, GradeDict, InfoDict) ### Pre v.s. Post Clerk Summary Comments (UIM v.s. NONUIM)

------------Pre------------
for UIM group:  -0.054481921623453604 with p-value 0.029418863931041286
for NonUIM group:  -0.052482101384745496 with p-value 0.000580748973825976
------------Post------------
for UIM group:  0.16564556179004974 with p-value 5.671603425515939e-13
for NonUIM group:  0.17308713645972776 with p-value 3.726846593819146e-27
------------UIM v.s. NONUIM------------
Pre-changing, UIM group sg:  0.7655728903224804 NonUIM group sg:  0.7602013358163394
TtestResult(statistic=1.3999103334765501, pvalue=0.16159283938244165, df=5890.0)
Post-changing, UIM group sg:  0.628715717611154 NonUIM group sg:  0.6353791578407603
TtestResult(statistic=-1.153496560427026, pvalue=0.2487549938642487, df=5700.0)


In [81]:
def general_SubI_Pre_Post_Corr(LetterGrade, SentiGrade, GradeDict):
    pre_LG = []
    pre_SG = []
    post_LG = []
    post_SG = []
    for deid in studentInfo.index.values:
            if deid in LetterGrade and deid in SentiGrade and LetterGrade[deid] in GradeDict:
                if studentInfo.loc[deid]['core_year'] in ['2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018']:
                    pre_LG.append(GradeDict[LetterGrade[deid]])
                    pre_SG.append(SentiGrade[deid])
                else:
                    post_LG.append(GradeDict[LetterGrade[deid]])
                    post_SG.append(SentiGrade[deid])
    
    resPre = stats.spearmanr(pre_SG, pre_LG)
    resPost= stats.spearmanr(post_SG, post_LG)
    
    print("Corr for pre is ", resPre.statistic, "with p-value: ", resPre.pvalue)
    print("Corr for post is ", resPost.statistic, "with p-value: ", resPost.pvalue)
    
    print("avg. pre SG is: ", sum(pre_SG) / len(pre_SG), "avg. post SG is: ", sum(post_SG) / len(post_SG))
    print(stats.ttest_ind(pre_SG, post_SG))

In [82]:
general_SubI_Pre_Post_Corr(subILetterGradeCC, subISentiGradeCC, SubIGradeDict) # for subI constructive comments

Corr for pre is  0.05989805063847228 with p-value:  0.14313216181935343
Corr for post is  0.1219483341231052 with p-value:  0.0018113410456006617
avg. pre SG is:  0.35235809233253285 avg. post SG is:  0.4552693958953031
TtestResult(statistic=-6.236114555900398, pvalue=6.126599551397478e-10, df=1249.0)


In [83]:
general_SubI_Pre_Post_Corr(subILetterGrade, subISentiGrade, SubIGradeDict) # for subI summary comments

Corr for pre is  -0.05967384157282337 with p-value:  0.10622540393157334
Corr for post is  -0.06765384207918713 with p-value:  0.08431933319145063
avg. pre SG is:  0.7062596108493456 avg. post SG is:  0.627896440852361
TtestResult(statistic=11.68980837120701, pvalue=3.572608164239971e-30, df=1384.0)


In [84]:
### Pre v.s. Post Clerk Constructive Comments (UIM v.s. NONUIM)
def partial_SubI_Pre_Post_Corr(LetterGrade, SentiGrade, GradeDict):
    preLG_UIM = []
    preSG_UIM = []
    preLG_NONUIM = []
    preSG_NONUIM = []
    postLG_UIM = []
    postSG_UIM = []
    postLG_NONUIM = []
    postSG_NONUIM = []
    for deid in studentInfo.index.values:
            if deid in LetterGrade and deid in SentiGrade and LetterGrade[deid] in GradeDict:
                if studentInfo.loc[deid]['core_year'] in ['2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018']:
                    if studentInfo.loc[deid]['isURM'] == 1 :
                        preLG_UIM.append(GradeDict[LetterGrade[deid]])
                        preSG_UIM.append(SentiGrade[deid])
                    else:
                        preLG_NONUIM.append(GradeDict[LetterGrade[deid]])
                        preSG_NONUIM.append(SentiGrade[deid])
                    
                else:
                    if studentInfo.loc[deid]['isURM'] == 1 :
                        postLG_UIM.append(GradeDict[LetterGrade[deid]])
                        postSG_UIM.append(SentiGrade[deid])
                    else:
                        postLG_NONUIM.append(GradeDict[LetterGrade[deid]])
                        postSG_NONUIM.append(SentiGrade[deid])

    resPreUIM = stats.spearmanr(preSG_UIM, preLG_UIM)
    resPreNONUIM = stats.spearmanr(preSG_NONUIM, preLG_NONUIM)
    
    resPostUIM = stats.spearmanr(postSG_UIM, postLG_UIM)
    resPostNONUIM = stats.spearmanr(postSG_NONUIM, postLG_NONUIM)
    
    print("------------Pre------------")
    print("for UIM group: ", resPreUIM.statistic, "with p-value", resPreUIM.pvalue)
    print("for NonUIM group: ", resPreNONUIM.statistic, "with p-value", resPreNONUIM.pvalue)
    print("------------Post------------")
    print("for UIM group: ", resPostUIM.statistic, "with p-value", resPostUIM.pvalue)
    print("for NonUIM group: ", resPostNONUIM.statistic, "with p-value", resPostNONUIM.pvalue)
    
    print("------------UIM v.s. NONUIM------------")
    print("Pre-changing, UIM group sg: ", sum(preSG_UIM) / len(preSG_UIM), "NonUIM group sg: ", sum(preSG_NONUIM) / len(preSG_NONUIM))
    print(stats.ttest_ind(preSG_UIM, preSG_NONUIM))
    
    print("Post-changing, UIM group sg: ", sum(postSG_UIM) / len(postSG_UIM),"NonUIM group sg: ", sum(postSG_NONUIM) / len(postSG_NONUIM))
    print(stats.ttest_ind(postSG_UIM, postSG_NONUIM))

In [85]:
partial_SubI_Pre_Post_Corr(subILetterGradeCC, subISentiGradeCC, SubIGradeDict) 

------------Pre------------
for UIM group:  0.08283709969155097 with p-value 0.2799932821549034
for NonUIM group:  0.05184617275170241 with p-value 0.2851026900313913
------------Post------------
for UIM group:  0.19515277348454188 with p-value 0.00493871790006929
for NonUIM group:  0.09607130213199067 with p-value 0.04257033227001538
------------UIM v.s. NONUIM------------
Pre-changing, UIM group sg:  0.35881944228206814 NonUIM group sg:  0.34975539399220495
TtestResult(statistic=0.35856907632964574, pvalue=0.7200442455562792, df=597.0)
Post-changing, UIM group sg:  0.49290323532278046 NonUIM group sg:  0.4378869498817149
TtestResult(statistic=2.168101902433537, pvalue=0.030513067884663743, df=650.0)


In [86]:
partial_SubI_Pre_Post_Corr(subILetterGrade, subISentiGrade, SubIGradeDict) 

------------Pre------------
for UIM group:  -0.05198087852443886 with p-value 0.4681857667491347
for NonUIM group:  -0.05965188176668729 with p-value 0.1674829608830123
------------Post------------
for UIM group:  -0.10532757497645044 with p-value 0.1318797737503881
for NonUIM group:  -0.04648611236287194 with p-value 0.32733230233110655
------------UIM v.s. NONUIM------------
Pre-changing, UIM group sg:  0.7097483166150409 NonUIM group sg:  0.7049797690693795
TtestResult(statistic=0.49436993469824336, pvalue=0.6211932696865877, df=732.0)
Post-changing, UIM group sg:  0.6282797643425357 NonUIM group sg:  0.6277193900923256
TtestResult(statistic=0.04967588244007862, pvalue=0.960395936047932, df=650.0)
